## Why is Community Engagement Rank a good choice for Farcaster?

Community Engagement Rank is an excellent choice for analyzing the Farcaster ecosystem for several reasons:

**1. Decentralized Social Network**: Farcaster is a decentralized social protocol, making engagement metrics crucial for understanding user interactions and influence.

**2. User Influence**: It helps identify the most influential and active users within the Farcaster community.

**3. Content Quality**: By analyzing reactions and mentions, we can gauge the quality and impact of content shared on the platform.

**4. Network Growth**: It provides insights into how the Farcaster network is growing and evolving over time.



In [ ]:
#Install necessary package
!pip install openrank_sdk 

## OpenRank: A Brief Overview

OpenRank is an open-source implementation of the EigenTrust algorithm, designed to compute trust scores in decentralized networks. It's particularly useful for analyzing and ranking entities in Web3 ecosystems, such as blockchain networks, decentralized social media platforms, and other peer-to-peer systems.

Key features of OpenRank include:

**1. Decentralized Trust Computation**: It calculates trust scores without relying on a central authority, making it ideal for decentralized networks.

**2. Customizable**: It allows for the incorporation of pre-existing trust information (pretrust) and can be adapted to various types of interaction data.

**3. Scalability**: OpenRank is designed to handle large-scale networks with millions of nodes and interactions.

OpenRank is a powerful algorithm that operates on reputation graph data, which is constructed from peer-to-peer trust signals within a network. At its core, OpenRank interprets and analyzes complex webs of trust relationships, answering the fundamental question: "Who trusts whom, by how much, and for what purpose?"

In this context:

    1. Who (The Truster): Represents the entity extending trust. This could be a user, a node, or any participant in the network capable of expressing trust.

    2. Whom (The Trustee): Denotes the recipient of trust. This is the entity being evaluated or trusted within the network.

    3. How Much: Quantifies the degree or intensity of trust from the truster to the trustee. This could be represented as a numerical value or weight.

    4. What (The Context): Specifies the nature or type of trust being extended. This crucial element provides context, defining the role or specific domain in which the trust is relevant.


In the context of our Community Engagement Rank for Farcaster, OpenRank will use the interaction data (reactions and mentions) to compute trust scores for users. These scores will reflect each user's influence and engagement level within the Farcaster community, based on how often they're interacted with and mentioned by others.

The OpenRank algorithm takes two main inputs:

    1. Localtrust: This is derived from the direct interactions between users, which in our case comes from the SQL query results showing reactions and mentions.

    2. Pretrust: This is a set of nodes (users) that are considered trustworthy from the outset. For Farcaster, this could be based on verified users or other criteria. We will be using Power Badge users from warpcast platform.

By applying OpenRank to our Farcaster data, we can obtain a nuanced and sybil-resistant ranking of users based on their community engagement, providing valuable insights into the dynamics of the Farcaster ecosystem.

## Power Badges

Power badges are awarded to active, engaging, and non-spammy users on Warpcast. Benefits include:

    1. Profile badge
    2. Higher search ranking
    3. Access to restricted channels
    4. Increased follow recommendations

Developers can access the list of badge holders via API for various purposes.

The selection process is based on a PageRank-like system considering:

    1. Recent activity
    2. Engagement from other power users
    3. Absence of spam flags

To increase chances of earning a power badge:

    1. Use Farcaster regularly
    2. Create content that appeals to power users
    3. Avoid posting spam or off-topic content

This system aims to recognize and reward valuable contributors to the Warpcast community.

## LocalTrust

Our Local Trust query for Farcaster focuses on two main types of interactions:

    1. Reactions: When a user reacts to another user's cast (post).
    2. Mentions: When a user mentions another user in their cast.

The query aggregates these interactions over the past 30 days to create a recent engagement score. We use the OpenRank algorithm to calculate the final rank based on these interactions.

We will use Farcaster dataset from Open Source Observer hosted on big query. 
You can either run the query directly from the notebook (you will need gcp credentials for that) or copy the query and paste in the big query editor and export the result as csv file. Remember to replace the "YOUR_PROJECT_NAME" with your GCP project name.

   
   
    WITH reactions AS (
        SELECT 
            FID,
            CAST(TARGET_CAST_FID AS STRING) AS interaction_id,
            COUNT(*) AS interactions
        FROM 
            {YOUR_PROJECT_NAME}.farcaster.reactions
        WHERE 
            TIMESTAMP >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 30 DAY)
            AND DELETED_AT IS NULL
        GROUP BY 
            FID, TARGET_CAST_FID
    ),
    mentions AS (
        SELECT 
            FID,
            JSON_VALUE(m, '$') AS interaction_id,
            COUNT(*) AS interactions
        FROM 
            {YOUR_PROJECT_NAME}.farcaster.casts,
            UNNEST(JSON_EXTRACT_ARRAY(MENTIONS)) AS m
        WHERE 
            TIMESTAMP >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 30 DAY)
            AND DELETED_AT IS NULL
        GROUP BY 
            FID, JSON_VALUE(m, '$')
    )
    SELECT fid, interaction_id, interactions
    FROM reactions
    UNION ALL
    SELECT fid, interaction_id, interactions
    FROM mentions
    ORDER BY interactions DESC
    LIMIT 100000



## Explaination of the LocalTrust Query

This query is designed to aggregate and combine interaction data from two different sources within open source observer's farcaster dataset hosted on BigQuery dataset. It uses Common Table Expressions (CTEs) to organize the data before combining the results. Here's a detailed breakdown of each part of the query:

### CTE: `reactions`

1. **Purpose**: 
   - This CTE aggregates interaction data from the `reactions` table.

2. **SELECT Clause**:
   - `FID`: The unique identifier for the user or entity initiating the interaction.
   - `CAST(TARGET_CAST_FID AS STRING) AS interaction_id`: Converts the target interaction ID to a string for consistent data type handling.
   - `COUNT(*) AS interactions`: Counts the number of interactions for each `FID` and `TARGET_CAST_FID` pair.

3. **FROM Clause**:
   - `{YOUR_PROJECT_NAME}.farcaster.reactions`: Specifies the table containing reaction data. #project name

4. **WHERE Clause**:
   - `TIMESTAMP >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 30 DAY)`: Filters records to include only those from the last 30 days.
   - `DELETED_AT IS NULL`: Ensures only active (non-deleted) records are included.

5. **GROUP BY Clause**:
   - Groups the results by `FID` and `TARGET_CAST_FID` to aggregate the interaction counts.

### CTE: `mentions`

1. **Purpose**: 
   - This CTE aggregates mention data from the `casts` table.

2. **SELECT Clause**:
   - `FID`: The unique identifier for the user or entity making the mention.
   - `CAST(JSON_VALUE(m) AS STRING) AS interaction_id`: Extracts and casts each mention from the JSON array to a string.
   - `COUNT(*) AS interactions`: Counts the number of mentions for each `FID` and mention.

3. **FROM Clause**:
   - `{YOUR_PROJECT_NAME}.farcaster.casts`: Specifies the table containing cast data.
   - `UNNEST(JSON_EXTRACT_ARRAY(mentions)) AS m`: Flattens the JSON array of mentions into individual elements for processing.

4. **WHERE Clause**:
   - `TIMESTAMP >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 30 DAY)`: Filters records to include only those from the last 30 days.
   - `DELETED_AT IS NULL`: Ensures only active (non-deleted) records are included.

5. **GROUP BY Clause**:
   - Groups the results by `FID` and the extracted mention to aggregate the mention counts.

### Final SELECT and UNION ALL

1. **SELECT Clause**:
   - Combines the results from both CTEs, selecting `fid`, `interaction_id`, and `interactions`.

2. **UNION ALL**:
   - Combines the results from the `reactions` and `mentions` CTEs into a single result set. `UNION ALL` is used to include all records from both CTEs without removing duplicates.

3. **ORDER BY Clause**:
   - Orders the combined results by the number of interactions in descending order, so the most interacted-with items appear first.

4. **LIMIT Clause**:
   - Limits the final result set to 100,000 records to manage the size of the output.

### Overall Purpose

The query is designed to provide a comprehensive view of interactions and mentions over the past 30 days, combining data from two different sources and presenting it in a unified format. This can be useful for analyzing user engagement or activity trends within the dataset.

# Run OpenRank on Farcaster data

In [1]:
## let read the data
import pandas as pd

localtrust = pd.read_csv("LocalTrust.csv")
localtrust.head()

,fid,interaction_id,interactions
0,403507,781098,87236
1,538721,503116,8984
2,284439,8446,5764
3,405154,244062,3940
4,460287,272116,3866


In [2]:
localtrust['fid']=localtrust['fid'].astype(str)
localtrust['interaction_id']=localtrust['interaction_id'].astype(str)

In [3]:
localtrust = localtrust[localtrust['fid'] != localtrust['interaction_id']]
localtrust = localtrust[localtrust['interactions']>10]

In [4]:
localtrust.dtypes

fid               object
interaction_id    object
interactions       int64
dtype: object

## PreTrust

As said earlier, we will be use power badge users.

In [12]:
#we will user warpcast api to get the current power badge users.
!curl https://api.warpcast.com/v2/power-badge-users | jq > power-badge.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11937  100 11937    0     0  19160      0 --:--:-- --:--:-- --:--:-- 19129


In [5]:
import json

with open("power-badge.json", "r") as f:
    powerBadgeUsers = json.load(f)

In [6]:
pretrust = [x for x in powerBadgeUsers['result']['fids']]

## OpenRank

In [7]:
#let's select only those power badge user who are in localtrust
all_fids = set(localtrust['fid']).union(set(localtrust['interaction_id']))
pretrust = [str(x) for x in pretrust if x in all_fids]

In [8]:
# create the graphs from LocalTrust and preTrust

localtrust_graph = [{'i': row['fid'], 'j':row['interaction_id'], 'v':row['interactions']} 
                                        for i, row in localtrust.iterrows()]
pretrust_graph = [{'i':fid, 'v':10000} for fid in pretrust]

In [ ]:
# Use openrank
from openrank_sdk import EigenTrust

a = EigenTrust()
farcaster_user_rank = a.run_eigentrust(localtrust_graph, pretrust_graph)
farcaster_user_rank[:10]